# Exercise PPM Part 2 - Small dataset

Imports

In [86]:
import datetime

import pandas as pd

Import an event log as a pandas dataframe

In [87]:
df = pd.read_csv("Event_Logs/small3insert.csv")
df

,Case,Event,Activity,Timestamp,Resource,Resource_failure_rate,Resource_Pass/Fail,order,resource_anomaly_type,resource_parameter,trace_change_resource,variant_num
0,case_0,9367,Activity A,1970-01-01 09:00:00,Resource_Group2_res_1,0.026984,0,1,normal,NaN,0,var_358
1,case_0,9368,Activity B,1970-01-01 10:00:00,Resource_Group0_res_0,0.018454,0,2,normal,NaN,0,var_358
2,case_0,9369,Activity C,1970-01-01 11:00:00,Resource_Group1_res_0,0.002551,0,3,normal,NaN,0,var_358
3,case_0,9370,Activity D,1970-01-01 12:00:00,Resource_Group2_res_2,0.029775,0,4,normal,NaN,0,var_358
4,case_0,9371,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,0.018454,0,5,normal,NaN,0,var_358
...,...,...,...,...,...,...,...,...,...,...,...,...
45842,case_999,31364,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,0.018454,0,5,normal,NaN,0,var_171
45843,case_999,31365,Activity K,1970-01-01 14:00:00,Resource_Group1_res_1,0.007068,0,6,normal,NaN,0,var_171
45844,case_999,31366,Activity Q,1970-01-01 15:00:00,Resource_Group0_res_0,0.018454,0,7,normal,NaN,0,var_171
45845,case_999,31367,Activity L,1970-01-01 16:00:00,Resource_Group0_res_0,0.018454,0,8,normal,NaN,0,var_171


Create an event log that retains only the attributes caseid, activity, timestamp, resource and outcome.

In [88]:
df = df.rename(columns={"Case":"caseid", "Activity":"activity", "Timestamp":"ts", "Resource":"resource", "resource_anomaly_type":"outcome"})
df_2 = df[["caseid", "activity", "ts", "resource", "outcome"]].copy()
df_2

,caseid,activity,ts,resource,outcome
0,case_0,Activity A,1970-01-01 09:00:00,Resource_Group2_res_1,normal
1,case_0,Activity B,1970-01-01 10:00:00,Resource_Group0_res_0,normal
2,case_0,Activity C,1970-01-01 11:00:00,Resource_Group1_res_0,normal
3,case_0,Activity D,1970-01-01 12:00:00,Resource_Group2_res_2,normal
4,case_0,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,normal
...,...,...,...,...,...
45842,case_999,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,normal
45843,case_999,Activity K,1970-01-01 14:00:00,Resource_Group1_res_1,normal
45844,case_999,Activity Q,1970-01-01 15:00:00,Resource_Group0_res_0,normal
45845,case_999,Activity L,1970-01-01 16:00:00,Resource_Group0_res_0,normal


 Create a function to extract the prefixes of length L from the event log.

In [89]:
import numpy as np
def create_prefix(df, L):
    res0 = df.sort_values(by=['caseid','ts']).groupby('caseid', as_index=False)['activity'].apply(list)    
    res1 = df.sort_values(by=['caseid','ts']).groupby('caseid', as_index=False)['ts'].apply(list)
    res2 = df.sort_values(by=['caseid','ts']).groupby('caseid', as_index=False)['resource'].apply(list)
    
    res = pd.merge(res0, pd.merge(res1, res2, on="caseid") , on="caseid")
    
    event_num = df.groupby('caseid')['activity'].count().to_frame('event_number')
    res = pd.merge(res, event_num, on="caseid")
    
    res[res["event_number"] < L] = np.nan
    res = res.dropna()
    
    def function(row, label):
        activity_array = row[label][0:L]
        return activity_array
    
    for c in ["activity", "ts", "resource"]:
        res[c] = res.apply(lambda row: function(row, c), axis=1)
    
    res = res.drop(columns=["event_number"])
    return res
    

Create a function that does aggregation encoding of a prefix extracted from the event log

In [90]:
x = create_prefix(df, 4)
x

,caseid,activity,ts,resource
0,case_0,"[Activity A, Activity B, Activity C, Activity D]","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_1, Resource_Group0_res_0,..."
1,case_1,"[Activity A, Activity B, Activity C, Activity G]","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_0, Resource_Group0_res_0,..."
2,case_10,"[Activity A, Activity B, Activity C, Activity D]","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_1, Resource_Group0_res_0,..."
3,case_100,"[Activity A, Activity B, Activity C, Activity D]","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_1, Resource_Group0_res_0,..."
4,case_1000,"[Activity A, Activity B, Activity C, Activity D]","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_0, Resource_Group0_res_0,..."
...,...,...,...,...
4995,case_995,"[Activity A, Activity B, Activity C, Activity D]","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_0, Resource_Group0_res_0,..."
4996,case_996,"[Activity A, Activity B, Activity C, Activity D]","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_2, Resource_Group0_res_0,..."
4997,case_997,"[Activity A, Activity B, Activity C, Activity D]","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_0, Resource_Group0_res_0,..."
4998,case_998,"[Activity A, Activity B, Activity C, Activity D]","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_0, Resource_Group0_res_0,..."


In [203]:
from datetime import *

def one_hot(x, label):
    temp = []
    for element in x[label]:
        for e in element:
            temp.append(e)
            
    df = pd.DataFrame(temp)
    unique_val = df[0].unique()
    
    result = x[["caseid"]].copy()
    for e in unique_val:
        result[e] = 0
    
    for index, row in x.iterrows():
        for activity in row[label]:
            result[activity].iloc[index] += 1
    
    return result

def average_label(df, label):
    result = x[["caseid"]].copy()
    result["avg_"+label] = 0
    for index, row in df.iterrows():
        result["avg_" + label].iloc[index] = pd.to_datetime(row[label][-1]) - pd.to_datetime(row[label][0])
    return result


In [204]:
average_label(x, "ts")

/var/folders/76/2m87n1zs4352j9ydg5g8h8dc0000gn/T/ipykernel_51254/817000814.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["avg_" + label].iloc[index] = pd.to_datetime(row[label][-1]) - pd.to_datetime(row[label][0])
/var/folders/76/2m87n1zs4352j9ydg5g8h8dc0000gn/T/ipykernel_51254/817000814.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0 days 03:00:00' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result["avg_" + label].iloc[index] = pd.to_datetime(row[label][-1]) - pd.to_datetime(row[label][0])


,caseid,avg_ts
0,case_0,0 days 03:00:00
1,case_1,0 days 02:07:35
2,case_10,0 days 03:00:00
3,case_100,0 days 03:00:00
4,case_1000,0 days 03:00:00
...,...,...
4995,case_995,0 days 03:00:00
4996,case_996,0 days 03:00:00
4997,case_997,0 days 03:00:00
4998,case_998,0 days 03:00:00


In [206]:
one_hot(x, "activity")

/var/folders/76/2m87n1zs4352j9ydg5g8h8dc0000gn/T/ipykernel_51254/817000814.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[activity].iloc[index] += 1
/var/folders/76/2m87n1zs4352j9ydg5g8h8dc0000gn/T/ipykernel_51254/817000814.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[activity].iloc[index] += 1
/var/folders/76/2m87n1zs4352j9ydg5g8h8dc0000gn/T/ipykernel_51254/817000814.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

,caseid,Activity A,Activity B,Activity C,Activity D,Activity G,Activity O,Activity M,Activity S,Activity R,Activity K,Activity Q,Activity N,Activity L,Activity I,Activity J,Activity P,Activity H,Activity T
0,case_0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,case_1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,case_10,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,case_100,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,case_1000,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,case_995,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4996,case_996,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4997,case_997,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4998,case_998,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [207]:
one_hot(x, "resource")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,caseid,Resource_Group2_res_1,Resource_Group0_res_0,Resource_Group1_res_0,Resource_Group2_res_2,Resource_Group2_res_0,Resource_Group1_res_1
0,case_0,1,1,1,1,0,0
1,case_1,1,1,0,0,1,1
2,case_10,1,1,1,0,1,0
3,case_100,1,1,1,0,1,0
4,case_1000,1,1,1,0,1,0
...,...,...,...,...,...,...,...
4995,case_995,0,1,1,1,1,0
4996,case_996,0,1,0,1,1,1
4997,case_997,0,1,0,0,2,1
4998,case_998,1,1,0,0,1,1


In [208]:
def aggregation_enc(x):
    res0 = one_hot(x, "activity")
    res1 = one_hot(x, "resource")
    res2 = average_label(x, "ts")
    
    res = pd.merge(res2, pd.merge(res1, res0), on="caseid")
    
    return res

In [209]:
aggregation_enc(x)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,caseid,avg_ts,Resource_Group2_res_1,Resource_Group0_res_0,Resource_Group1_res_0,Resource_Group2_res_2,Resource_Group2_res_0,Resource_Group1_res_1,Activity A,Activity B,...,Activity R,Activity K,Activity Q,Activity N,Activity L,Activity I,Activity J,Activity P,Activity H,Activity T
0,case_0,0 days 03:00:00,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,case_1,0 days 02:07:35,1,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,case_10,0 days 03:00:00,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,case_100,0 days 03:00:00,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,case_1000,0 days 03:00:00,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,case_995,0 days 03:00:00,0,1,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4996,case_996,0 days 03:00:00,0,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4997,case_997,0 days 03:00:00,0,1,0,0,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4998,case_998,0 days 03:00:00,1,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
